In [1]:
import gym
import numpy as np
import random
import math
from time import sleep


## Initialize the "Cart-Pole" environment
env = gym.make('CartPole-v0')

## Defining the environment related constants

# Number of discrete states (bucket) per state dimension
NUM_BUCKETS = (1, 1, 6, 3)  # (x, x', theta, theta')
# Number of discrete actions
NUM_ACTIONS = env.action_space.n # (left, right)
# Bounds for each discrete state
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))
STATE_BOUNDS[1] = [-0.5, 0.5]
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]
# Index of the action
ACTION_INDEX = len(NUM_BUCKETS)

## Creating a Q-Table for each state-action pair
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))

## Learning related constants
MIN_EXPLORE_RATE = 0.01
MIN_LEARNING_RATE = 0.1

## Defining the simulation related constants
NUM_EPISODES = 1000
MAX_T = 250
STREAK_TO_END = 120
SOLVED_T = 199
DEBUG_MODE = True

def simulate():

    ## Instantiating the learning related parameters
    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)
    discount_factor = 0.99  # since the world is unchanging

    num_streaks = 0

    for episode in range(NUM_EPISODES):

        # Reset the environment
        obv = env.reset()

        # the initial state
        state_0 = state_to_bucket(obv)

        for t in range(MAX_T):
            # env.render()

            # Select an action
            action = select_action(state_0, explore_rate)

            # Execute the action
            obv, reward, done, _ = env.step(action)

            # Observe the result
            state = state_to_bucket(obv)

            # Update the Q based on the result
            best_q = np.amax(q_table[state])
            q_table[state_0 + (action,)] += learning_rate*(reward + discount_factor*(best_q) - q_table[state_0 + (action,)])

            # Setting up for the next iteration
            state_0 = state

            # Print data
            if (DEBUG_MODE):
                print("\nEpisode = %d" % episode)
                print("t = %d" % t)
                print("Action: %d" % action)
                print("State: %s" % str(state))
                print("Reward: %f" % reward)
                print("Best Q: %f" % best_q)
                print("Explore rate: %f" % explore_rate)
                print("Learning rate: %f" % learning_rate)
                print("Streaks: %d" % num_streaks)

                print("")

            if done:
               print("Episode %d finished after %f time steps" % (episode, t))
               if (t >= SOLVED_T):
                   num_streaks += 1
               else:
                   num_streaks = 0
               break

            #sleep(0.25)

        # It's considered done when it's solved over 120 times consecutively
        if num_streaks > STREAK_TO_END:
            break

        # Update parameters
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)


def select_action(state, explore_rate):
    # Select a random action
    if random.random() < explore_rate:
        action = env.action_space.sample()
    # Select the action with the highest q
    else:
        action = np.argmax(q_table[state])
    return action


def get_explore_rate(t):
    return max(MIN_EXPLORE_RATE, min(1, 1.0 - math.log10((t+1)/25)))

def get_learning_rate(t):
    return max(MIN_LEARNING_RATE, min(0.5, 1.0 - math.log10((t+1)/25)))

def state_to_bucket(state):
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
        else:
            # Mapping the state bounds to the bucket array
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            offset = (NUM_BUCKETS[i]-1)*STATE_BOUNDS[i][0]/bound_width
            scaling = (NUM_BUCKETS[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i] - offset))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)

if __name__ == "__main__":
    simulate()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.

Episode = 0
t = 0
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 0.000000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 1
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 0.500000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 2
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 0.747500
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 3
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 1.243763
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 4
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 1.365662
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 5
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 0.000000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 6
Action: 1
State: (0, 0, 3, 0)


Reward: 1.000000
Best Q: 4.789500
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 2
t = 7
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 0.998750
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 2
t = 8
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 0.998750
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 2
t = 9
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 1.493756
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 2
t = 10
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 1.490644
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 2
t = 11
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 1.490644
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 2
t = 12
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 1.737244
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 2
t = 13
Action: 1
State: (0, 0, 2, 0)
Re

Best Q: 6.529305
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 17
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 6.638023
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 18
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 1.986287
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 19
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 6.529305
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 20
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 6.529305
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 21
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 6.996658
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 22
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 7.461675
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 23
Action: 0
State: (0, 0, 3, 2)
Reward: 1.000000


Reward: 1.000000
Best Q: 13.779723
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 14
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 13.779723
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 15
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 9.066420
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 16
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 13.779723
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 17
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 11.854173
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 18
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 11.854173
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 19
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 9.067207
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 20
Action: 1
State:

Best Q: 13.463267
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 18
t = 11
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 13.463267
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 18
t = 12
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 13.719385
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 18
t = 13
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 14.022729
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 18
t = 14
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 14.300944
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 18
t = 15
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 14.729439
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 18
t = 16
Action: 1
State: (0, 0, 1, 0)
Reward: 1.000000
Best Q: 5.242170
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 18
t = 17
Action: 1
State: (0, 0, 1, 0)
Rew

...


Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 89.467635
Explore rate: 0.172631
Learning rate: 0.172631
Streaks: 0


Episode = 168
t = 10
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 89.467635
Explore rate: 0.172631
Learning rate: 0.172631
Streaks: 0


Episode = 168
t = 11
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 78.695239
Explore rate: 0.172631
Learning rate: 0.172631
Streaks: 0


Episode = 168
t = 12
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 78.695239
Explore rate: 0.172631
Learning rate: 0.172631
Streaks: 0


Episode = 168
t = 13
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 78.732017
Explore rate: 0.172631
Learning rate: 0.172631
Streaks: 0


Episode = 168
t = 14
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 89.602484
Explore rate: 0.172631
Learning rate: 0.172631
Streaks: 0


Episode = 168
t = 15
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 89.602484
Explore rate: 0.172631
Learning rate: 0.172631
Streaks: 0




Episode = 169
t = 35
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 89.836140
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 36
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 88.513587
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 37
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 89.630769
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 38
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.128920
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 39
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.128920
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 40
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.149107
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 41
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.169260
Explore rate: 0.170053
Learning rate

t = 135
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.768314
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 136
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.768314
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 137
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 88.849891
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 138
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.816999
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 139
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.816999
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 140
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 88.816999
Explore rate: 0.170053
Learning rate: 0.170053
Streaks: 0


Episode = 169
t = 141
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 85.973965
Explore rate: 0.170053
Learning rate: 0.17005

Reward: 1.000000
Best Q: 86.898076
Explore rate: 0.164944
Learning rate: 0.164944
Streaks: 0



KeyboardInterrupt: 

In [30]:
a = env.action_space
a.